In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**----------Load The Data---------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [19]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


----------

**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

-----------

**----------VP Tree Dyn---------------------**

In [9]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

----------

**------VP Tree Node Size Optimization For Rebuild------**

**------------Run 1---------------------**

In [10]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 35.16it/s]


-------------------------------------------------------------
Build Time: 16.2910 sec, Search Time: 0.8184 sec
Leaf Size: 100, Total Time: 17.1094 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 50.86it/s]


-------------------------------------------------------------
Build Time: 15.1912 sec, Search Time: 0.5835 sec
Leaf Size: 200, Total Time: 15.7747 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 9.7211 sec, Search Time: 0.4290 sec
Leaf Size: 9,800, Total Time: 10.1501 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 73.47it/s]


-------------------------------------------------------------
Build Time: 9.5571 sec, Search Time: 0.4208 sec
Leaf Size: 9,900, Total Time: 9.9779 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 71.29it/s]

-------------------------------------------------------------
Build Time: 9.7360 sec, Search Time: 0.4358 sec
Leaf Size: 10,000, Total Time: 10.1718 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 16.2910 sec, Search Time: 0.8184 sec, Total Time: 17.1094 sec
Leaf Size: 200, Build Time: 15.1912 sec, Search Time: 0.5835 sec, Total Time: 15.7747 sec
Leaf Size: 300, Build Time: 14.0534 sec, Search Time: 0.4634 sec, Total Time: 14.5168 sec
Leaf Size: 400, Build Time: 14.0888 sec, Search Time: 0.4774 sec, Total Time: 14.5661 sec
Leaf Size: 500, Build Time: 13.1494 sec, Search Time: 0.4001 sec, Total Time: 13.5494 sec
Leaf Size: 600, Build Time: 14.1160 sec, Search Time: 0.4024 sec, Total Time: 14.5184 sec
Leaf Size: 700, Build Time: 13.2012 sec, Search Time: 0.4045 sec, Total Time: 13.6057 sec
Leaf Size: 800, Build Time: 13.0640 sec, Search Time: 0.4028 sec, Total Time: 13.4669 sec
Leaf Size: 900, Build Time: 13.1822 sec, Search Time: 0.3987 sec, Total Time: 13.580

**---------------------Run 2---------------**

In [12]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 27.01it/s]


-------------------------------------------------------------
Build Time: 19.3755 sec, Search Time: 0.9937 sec
Leaf Size: 100, Total Time: 20.3692 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 43.29it/s]


-------------------------------------------------------------
Build Time: 18.3808 sec, Search Time: 0.6346 sec
Leaf Size: 200, Total Time: 19.0154 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 10.0783 sec, Search Time: 0.4518 sec
Leaf Size: 9,800, Total Time: 10.5301 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 66.50it/s]


-------------------------------------------------------------
Build Time: 9.9241 sec, Search Time: 0.4579 sec
Leaf Size: 9,900, Total Time: 10.3820 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 65.66it/s]

-------------------------------------------------------------
Build Time: 10.5881 sec, Search Time: 0.4600 sec
Leaf Size: 10,000, Total Time: 11.0481 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 19.3755 sec, Search Time: 0.9937 sec, Total Time: 20.3692 sec
Leaf Size: 200, Build Time: 18.3808 sec, Search Time: 0.6346 sec, Total Time: 19.0154 sec
Leaf Size: 300, Build Time: 16.7657 sec, Search Time: 0.4982 sec, Total Time: 17.2639 sec
Leaf Size: 400, Build Time: 16.9292 sec, Search Time: 0.4935 sec, Total Time: 17.4227 sec
Leaf Size: 500, Build Time: 14.2610 sec, Search Time: 0.4350 sec, Total Time: 14.6961 sec
Leaf Size: 600, Build Time: 14.2889 sec, Search Time: 0.4761 sec, Total Time: 14.7650 sec
Leaf Size: 700, Build Time: 15.3535 sec, Search Time: 0.4299 sec, Total Time: 15.7834 sec
Leaf Size: 800, Build Time: 15.3674 sec, Search Time: 0.4632 sec, Total Time: 15.8306 sec
Leaf Size: 900, Build Time: 14.5443 sec, Search Time: 0.4418 sec, Total Time: 14.98

**-----Opt Node Size-----**

In [13]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [17.1094,15.7747,14.5168,14.5661,13.5494,14.5184,13.6057,13.4669,13.5809,
 12.8877,12.7385,12.6625,13.3501,12.8481,12.6083,13.2018,12.7802,12.7894,
 13.0572,11.6722,11.9766,11.7651,11.7929,11.9797,11.7539,12.0979,12.2875,
 12.0326,12.5356,12.1317,11.763 ,12.1051,11.894 ,12.3358,12.2144,12.0896,
 11.9255,11.9738,12.0608,10.9006,10.9407,10.8462,10.7868,11.4736,10.9835,
 13.7826,13.0601,13.4263,13.7283,13.7427,14.1992,14.8165,15.0837,13.332 ,
 15.1479,14.0749,13.3035,14.6356,13.6697,12.4246,12.8196,13.8846,14.0305,
 12.1462,14.015 ,12.8602,13.9494,14.5196,13.9844,12.3901,12.8188,13.3768,
 14.2965,11.8343,13.7111,13.6686,13.1867,13.4175,10.1723,10.4031,10.3153,
 10.3287,10.3568,10.2037,10.2538,10.072 ,10.0565,10.0838,10.1946,10.1194,
 10.2092,10.2309,10.2477,10.3526,10.2764,10.2479,10.3276,10.1501, 9.9779,
 10.1718]
    )

total_times_2 = np.array(
    [20.3692,19.0154,17.2639,17.4227,14.6961,14.765 ,15.7834,15.8306,14.9861,
 14.2672,14.0011,13.8845,13.7097,15.4627,13.7938,14.7043,14.0782,13.6641,
 15.2328,13.6943,12.6321,13.3016,13.0219,12.6082,12.5235,12.7394,12.7209,
 12.3684,12.9481,14.2878,12.6148,12.4792,12.5667,12.6876,12.4254,12.2664,
 12.8042,12.4535,12.3208,11.3218,11.6535,11.6772,11.5731,11.8063,11.4192,
 11.2864,11.2253,11.3872,11.194 ,11.287 ,11.5807,11.4863,11.5041,11.4543,
 11.3559,11.7927,11.1573,11.5362,11.3775,11.3551,11.3276,11.9256,11.5327,
 11.644 ,11.3713,11.0839,11.3804,11.2982,11.4743,11.4546,11.1049,11.5664,
 11.6021,11.816 ,11.528 ,11.425 ,11.1568,11.4459,10.3615,10.5733,10.2768,
 10.3435,10.3204,10.3542,10.2425,10.5758,10.2092,10.5146,10.2848,10.2691,
 10.3136,10.1667,10.4432,10.6311,10.4631,10.3317,10.3334,10.5301,10.382 ,
 11.0481]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 10.1328
Minimum found at leaf size: 8700


----------------------------

**-----------Epsilon Optimization For Rebuild---------**

**---------------Run 1---------------------**

In [14]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 8700
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 66.52it/s]


-------------------------------------------------------------
Build Time: 10.1120 sec, Search Time: 0.4500 sec
Epsilon: 0.0, Total Time: 10.5619 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 1353.83it/s]


-------------------------------------------------------------
Build Time: 3.6818 sec, Search Time: 0.0211 sec
Epsilon: 0.5, Total Time: 3.7029 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.4454 sec, Search Time: 0.0024 sec
Epsilon: 29.0, Total Time: 1.4478 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 41080.35it/s]


-------------------------------------------------------------
Build Time: 1.4839 sec, Search Time: 0.0021 sec
Epsilon: 29.5, Total Time: 1.4861 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 28387.84it/s]

-------------------------------------------------------------
Build Time: 1.4610 sec, Search Time: 0.0025 sec
Epsilon: 30.0, Total Time: 1.4635 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 10.1120 sec, Search Time: 0.4500 sec, Total Time: 10.5619 sec
Epsilon: 0.5, Build Time: 3.6818 sec, Search Time: 0.0211 sec, Total Time: 3.7029 sec
Epsilon: 1.0, Build Time: 3.3958 sec, Search Time: 0.0058 sec, Total Time: 3.4016 sec
Epsilon: 1.5, Build Time: 3.3058 sec, Search Time: 0.0030 sec, Total Time: 3.3088 sec
Epsilon: 2.0, Build Time: 3.3276 sec, Search Time: 0.0029 sec, Total Time: 3.3304 sec
Epsilon: 2.5, Build Time: 3.2462 sec, Search Time: 0.0031 sec, Total Time: 3.2492 sec
Epsilon: 3.0, Build Time: 2.7405 sec, Search Time: 0.0079 sec, Total Time: 2.7484 sec
Epsilon: 3.5, Build Time: 2.3486 sec, Search Time: 0.0035 sec, Total Time: 2.3520 sec
Epsilon: 4.0, Build Time: 2.2005 sec, Search Time: 0.0024 sec, Total Time: 2.2030 sec
Epsilon: 4.5, Build Time: 2.13

**-------------Run 2------------------------**

In [16]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 8700
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 82.23it/s]


-------------------------------------------------------------
Build Time: 10.0416 sec, Search Time: 0.3922 sec
Epsilon: 0.0, Total Time: 10.4337 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 1566.50it/s]


-------------------------------------------------------------
Build Time: 3.6219 sec, Search Time: 0.0188 sec
Epsilon: 0.5, Total Time: 3.6406 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.4298 sec, Search Time: 0.0020 sec
Epsilon: 29.0, Total Time: 1.4317 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 71331.70it/s]


-------------------------------------------------------------
Build Time: 1.4108 sec, Search Time: 0.0022 sec
Epsilon: 29.5, Total Time: 1.4130 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 36743.79it/s]

-------------------------------------------------------------
Build Time: 1.4512 sec, Search Time: 0.0031 sec
Epsilon: 30.0, Total Time: 1.4543 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 10.0416 sec, Search Time: 0.3922 sec, Total Time: 10.4337 sec
Epsilon: 0.5, Build Time: 3.6219 sec, Search Time: 0.0188 sec, Total Time: 3.6406 sec
Epsilon: 1.0, Build Time: 3.3670 sec, Search Time: 0.0052 sec, Total Time: 3.3722 sec
Epsilon: 1.5, Build Time: 3.4080 sec, Search Time: 0.0028 sec, Total Time: 3.4108 sec
Epsilon: 2.0, Build Time: 3.3650 sec, Search Time: 0.0034 sec, Total Time: 3.3685 sec
Epsilon: 2.5, Build Time: 3.1387 sec, Search Time: 0.0027 sec, Total Time: 3.1414 sec
Epsilon: 3.0, Build Time: 2.7361 sec, Search Time: 0.0087 sec, Total Time: 2.7448 sec
Epsilon: 3.5, Build Time: 2.3383 sec, Search Time: 0.0036 sec, Total Time: 2.3418 sec
Epsilon: 4.0, Build Time: 2.1703 sec, Search Time: 0.0026 sec, Total Time: 2.1728 sec
Epsilon: 4.5, Build Time: 2.17

----------------------------

**-----------Epsilon Optimization For Rebuild------------**

In [17]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [10.5619, 3.7029, 3.4016, 3.3088, 3.3304, 3.2492, 2.7484, 2.352 , 2.203 ,
  2.1332, 2.1935, 2.3036, 2.3606, 1.7734, 1.646 , 1.5467, 1.6785, 1.5804,
  1.5191, 1.5304, 1.6353, 1.4506, 1.4563, 1.4254, 1.4584, 1.4767, 1.4378,
  1.4618, 1.4047, 1.4064, 1.4433, 1.4097, 1.446 , 1.4548, 1.4085, 1.4561,
  1.4756, 1.4705, 1.4393, 1.4614, 1.4291, 1.4296, 1.4022, 1.4403, 1.4252,
  1.4105, 1.4223, 1.4111, 1.4916, 1.4643, 1.455 , 1.4416, 1.4109, 1.4552,
  1.4388, 1.4222, 1.4537, 1.4476, 1.4478, 1.4861, 1.4635]
    )

total_times_2 = np.array(
    [10.4337, 3.6406, 3.3722, 3.4108, 3.3685, 3.1414, 2.7448, 2.3418, 2.1728,
  2.1817, 2.163 , 2.1885, 2.1283, 1.7001, 1.625 , 1.5902, 1.6146, 1.6089,
  1.5012, 1.4762, 1.4067, 1.4269, 1.4748, 1.4277, 1.4572, 1.4652, 1.4186,
  1.4277, 1.4748, 1.407 , 1.4732, 1.4435, 1.4055, 1.4457, 1.3995, 1.4638,
  1.4454, 1.4109, 1.4632, 1.4007, 1.4239, 1.4143, 1.4546, 1.4524, 1.4713,
  1.4451, 1.4195, 1.4381, 1.4285, 1.4894, 1.4658, 1.5005, 1.4041, 1.4362,
  1.4798, 1.413 , 1.4341, 1.4278, 1.4317, 1.413 , 1.4543]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 1.4040
Minimum found at epsilon size: 17.0


**------Epsilon Optimization For Insertion------------**

**----------Run 1-----------------**

In [18]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 19.50it/s]


-------------------------------------------------------------
Build Time: 8.3136 sec, Search Time: 1.1919 sec
Epsilon: 0.0, Total Time: 9.5054 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:01<00:00, 19.95it/s]


-------------------------------------------------------------
Build Time: 8.2733 sec, Search Time: 1.1730 sec
Epsilon: 0.5, Total Time: 9.4463 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.2980 sec, Search Time: 1.0710 sec
Epsilon: 29.0, Total Time: 9.3690 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 22.75it/s]


-------------------------------------------------------------
Build Time: 8.2857 sec, Search Time: 1.0625 sec
Epsilon: 29.5, Total Time: 9.3482 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 22.49it/s]

-------------------------------------------------------------
Build Time: 8.2713 sec, Search Time: 1.0713 sec
Epsilon: 30.0, Total Time: 9.3426 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.3136 sec, Search Time: 1.1919 sec, Total Time: 9.5054 sec
Epsilon: 0.5, Build Time: 8.2733 sec, Search Time: 1.1730 sec, Total Time: 9.4463 sec
Epsilon: 1.0, Build Time: 8.2697 sec, Search Time: 1.2333 sec, Total Time: 9.5030 sec
Epsilon: 1.5, Build Time: 8.3534 sec, Search Time: 1.2472 sec, Total Time: 9.6006 sec
Epsilon: 2.0, Build Time: 8.4155 sec, Search Time: 1.2422 sec, Total Time: 9.6577 sec
Epsilon: 2.5, Build Time: 8.2077 sec, Search Time: 1.2535 sec, Total Time: 9.4612 sec
Epsilon: 3.0, Build Time: 8.3561 sec, Search Time: 1.1351 sec, Total Time: 9.4912 sec
Epsilon: 3.5, Build Time: 8.3790 sec, Search Time: 1.1520 sec, Total Time: 9.5310 sec
Epsilon: 4.0, Build Time: 8.2056 sec, Search Time: 1.1295 sec, Total Time: 9.3351 sec
Epsilon: 4.5, Build Time: 8.3188

**----------Run 2---------------**

In [20]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 18.49it/s]


-------------------------------------------------------------
Build Time: 8.5361 sec, Search Time: 1.2526 sec
Epsilon: 0.0, Total Time: 9.7887 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:01<00:00, 19.90it/s]


-------------------------------------------------------------
Build Time: 8.2855 sec, Search Time: 1.1867 sec
Epsilon: 0.5, Total Time: 9.4722 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.2900 sec, Search Time: 1.0722 sec
Epsilon: 29.0, Total Time: 9.3622 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 22.57it/s]


-------------------------------------------------------------
Build Time: 8.2554 sec, Search Time: 1.0696 sec
Epsilon: 29.5, Total Time: 9.3249 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 22.96it/s]

-------------------------------------------------------------
Build Time: 8.1662 sec, Search Time: 1.0536 sec
Epsilon: 30.0, Total Time: 9.2198 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.5361 sec, Search Time: 1.2526 sec, Total Time: 9.7887 sec
Epsilon: 0.5, Build Time: 8.2855 sec, Search Time: 1.1867 sec, Total Time: 9.4722 sec
Epsilon: 1.0, Build Time: 8.2214 sec, Search Time: 1.2445 sec, Total Time: 9.4660 sec
Epsilon: 1.5, Build Time: 8.3258 sec, Search Time: 1.2471 sec, Total Time: 9.5729 sec
Epsilon: 2.0, Build Time: 8.2401 sec, Search Time: 1.2483 sec, Total Time: 9.4884 sec
Epsilon: 2.5, Build Time: 8.2439 sec, Search Time: 1.2480 sec, Total Time: 9.4919 sec
Epsilon: 3.0, Build Time: 8.3786 sec, Search Time: 1.1361 sec, Total Time: 9.5147 sec
Epsilon: 3.5, Build Time: 8.2894 sec, Search Time: 1.1321 sec, Total Time: 9.4215 sec
Epsilon: 4.0, Build Time: 8.2594 sec, Search Time: 1.1394 sec, Total Time: 9.3988 sec
Epsilon: 4.5, Build Time: 8.2132

**--------Epsilon Optimization For Insert---------**

In [21]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [9.5054,9.4463,9.503 ,9.6006,9.6577,9.4612,9.4912,9.531 ,9.3351,9.4664,
 9.3925,9.496 ,9.4842,9.3012,9.328 ,9.3883,9.3198,9.3427,9.3546,9.1016,
 9.3573,9.3234,9.251 ,9.3493,9.4178,9.2667,9.3065,9.2641,9.3362,9.3375,
 9.3051,9.4923,9.3422,9.3534,9.5141,9.4266,9.2917,9.392 ,9.3685,9.3636,
 9.3106,9.3372,9.336 ,9.3772,9.3209,9.373 ,9.3177,9.3556,9.3173,9.2945,
 9.4277,9.3674,9.3533,9.3978,9.3271,9.3158,9.3428,9.2966,9.369 ,9.3482,
 9.3426]
    )

total_times_2 = np.array(
    [9.7887,9.4722,9.466 ,9.5729,9.4884,9.4919,9.5147,9.4215,9.3988,9.3564,
 9.4233,9.5958,9.4453,9.3759,9.4789,9.4235,9.2703,9.3665,9.3325,9.2462,
 9.4308,9.3309,9.3564,9.2538,9.3393,9.3986,9.4311,9.3743,9.5758,9.331 ,
 9.5381,9.4671,9.4259,9.408 ,9.4088,9.3423,9.3966,9.28  ,9.3276,9.3056,
 9.2777,9.3352,9.3735,9.2831,9.3886,9.3081,9.4847,9.4997,9.4737,9.5518,
 9.33  ,9.3018,9.3542,9.309 ,9.436 ,9.4113,9.2668,9.3371,9.3622,9.3249,
 9.2198]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.1739
Minimum found at epsilon size: 9.5


--------------------